In [2]:
from bs4 import BeautifulSoup
import re
from bs4 import Tag, NavigableString
from bs4.formatter import HTMLFormatter

In [3]:
with open(r"D:\Users\Dell\Documents\Web Projects\react\Plant Diseases\web-scraping\initial-extract\vegetable crops\mint.html",encoding="UTF-8") as fh:
    soup=BeautifulSoup(fh.read(),"html.parser")

In [4]:
d=soup.find("div",{"class":"col text-right"})

In [5]:
d.decompose()

In [6]:
soup.find("div",{"class":"col text-right"})

In [7]:
removable=soup.find_all("p")[-3:]
removable.append(soup.find_all("h3")[-1])

In [8]:
for tag in removable:
    tag.decompose()

In [9]:
def update_html(soup):
    with open("test.html","w",encoding="UTF-8") as fh:
        fh.write(str(soup.prettify(formatter=HTMLFormatter(indent=4))))

In [10]:
update_html(soup)

In [11]:
def CleanSoup(soup):
    content=clone_soup(soup)
    for tag in content.findAll(True): 
        if tag.name=='a':
            if tag.attrs.get("name",None):
                tag.attrs = {"name":tag.attrs["name"]}
            elif tag.attrs.get("href",None):
                tag.attrs = {"href":tag.attrs["href"]}
            else:
                tag.decompose()
        elif tag.name=='img':
            attrs=dict()
            for prop in ["width","height","alt","src"]:
                value=tag.attrs.get(prop,None)
                if value:
                    attrs[prop]=value
            tag.attrs=attrs
        else:
            tag.attrs = {}
    return content

In [12]:
def clone_soup(el):
    if isinstance(el, NavigableString):
        return type(el)(el)

    copy = Tag(None, el.builder, el.name, el.namespace, el.nsprefix)
    # work around bug where there is no builder set
    # https://bugs.launchpad.net/beautifulsoup/+bug/1307471
    copy.attrs = dict(el.attrs)
    for attr in ('can_be_empty_element', 'hidden'):
        setattr(copy, attr, getattr(el, attr))
    for child in el.contents:
        copy.append(clone_soup(child))
    return copy

In [13]:
update_html(CleanSoup(soup))
soup1=CleanSoup(soup)

In [14]:
for tag in soup1.findAll(True): 
    if tag.name=='a':
        if tag.attrs.get("name",None):
            tag.attrs = {"name":tag.attrs["name"]}
            tag.insert_before(soup.new_string("REPLACE"))

In [15]:
update_html(soup1)

In [16]:
pattern=r'REPLACE(?=<a.*?name="section.*?(?=a>))'

In [17]:
new_text=re.sub(pattern,"\n</section>\n<section>\n",str(soup1))

In [18]:
soup2=BeautifulSoup(new_text,"html.parser")

In [19]:
update_html(soup2)

In [20]:
last_section=soup2.find_all("section")[-1]
print(last_section)
last_section.decompose()

<section>
<a name="section9"></a>
</section>


In [21]:
soup2.div.unwrap()
soup2.div.unwrap()

<div></div>

In [22]:
update_html(soup2)

In [23]:
for tag in soup2.div.find_all("div",recursive=False):
    tag.div.unwrap()


In [24]:
soup2.div.find_all("div",recursive=False)[0].div.unwrap()

<div></div>

In [25]:
soup2.div.div.name="section"

In [26]:
update_html(soup2)

In [27]:
for section in soup2.findAll("section"):
    image=section.find("img")
    if image:
        image.attrs["alt"]=image.attrs.get("alt",image.find_previous("h3").text)
        parent=image.find_parent()
        if parent.name in ['p','li']:
            ul=image.find_previous("ul")
            ul.insert_before(image.extract())
            if parent.text:
                new_li=soup2.new_tag('li')
                new_li.append(soup2.new_string(str(parent.text)))
                ul.insert(0,new_li)
            parent.decompose()

In [28]:
for tag in soup2.findAll("strong"):
    tag.name="h4"

In [29]:
update_html(soup2)

In [30]:
def get_next_tag(soup):
    next_ele=soup
    while next_ele is not None:
        next_ele=next_ele.next_element
        if isinstance(next_ele,Tag):
            return next_ele
    return None

In [31]:
def get_previous_tag(soup):
    next_ele=soup
    while next_ele is not None:
        next_ele=next_ele.previous_element
        if isinstance(next_ele,Tag):
            return next_ele
    return None

In [32]:
# soup2.select("section ul + *")  # using css selectors

In [33]:
for li in soup2.findAll("li"):
    for h4 in li.find_all("h4"):
        h4.extract()
        li.string=str(h4.text+" "+li.text)     

In [34]:
update_html(soup2)

In [35]:
soup3=clone_soup(soup2)

In [36]:
update_html(soup3)

In [37]:
main_dict={}
for section in soup3.findAll("section")[1:]:
    disease=str(section.find("h3").text)
    d=dict()
    for h4 in section.findAll("h4"):
        content={}
        next_ele=get_next_tag(h4)
        if next_ele.name=="img":
            content["img"]={
                "src":next_ele.attrs["src"],
                "alt":next_ele.attrs.get("alt",disease)
            }
            next_ele=get_next_tag(next_ele)
        if next_ele.name=="ul":
            content["content"]=list(map(lambda x:x.text,next_ele.find_all("li")))
        d[str(h4.text)]=content

    main_dict[disease]=d     

In [38]:
for tag in soup3.findAll(["h4","ul"]):
        while not tag.parent.name == "section":
            tag.parent.unwrap()

In [39]:
import json
json_object = json.dumps(main_dict, indent = 4) 